In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [3]:
import nltk
nltk.download('wordnet') 
nltk.download('omw-1.4')  
nltk.download('stopwords') 

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
train_data = pd.read_csv('/kaggle/input/wec-task-1b/news_train.csv')
test_data = pd.read_csv('/kaggle/input/wec-task-1b/test.csv')

In [5]:
print("Missing values in training data:")
print(train_data.isnull().sum())

Missing values in training data:
ID                0
News_title        0
News_headline    76
Category          0
dtype: int64


In [6]:
train_data['News_headline'] = train_data['News_headline'].fillna('missing_headline')
test_data['News_headline'] = test_data['News_headline'].fillna('missing_headline')

In [7]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [8]:
def preprocess_text(text):
    if isinstance(text, str):
        # Lowercase the text
        text = text.lower()
        text = re.sub(r'\W+', ' ', text)
        text = re.sub(r'\d+', '', text)
        text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
    else:
        text = ''  
    return text

In [9]:
train_data['text'] = train_data['News_title'] + ' ' + train_data['News_headline']
test_data['text'] = test_data['News_title'] + ' ' + test_data['News_headline']

In [10]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
! unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/

In [11]:
train_data['clean_text'] = train_data['text'].apply(preprocess_text)
test_data['clean_text'] = test_data['text'].apply(preprocess_text)

In [12]:
train_data['title_length'] = train_data['News_title'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)
train_data['headline_length'] = train_data['News_headline'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)

test_data['title_length'] = test_data['News_title'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)
test_data['headline_length'] = test_data['News_headline'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)

In [13]:
category_mapping = {'Arts': 0, 'business': 1, 'humour': 2, 'politics': 3, 'sports': 4, 'tech': 5}
y_train = train_data['Category'].map(category_mapping)

In [14]:
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    train_data['clean_text'], y_train, test_size=0.2, random_state=42
)

In [15]:
# tfidf = TfidfVectorizer(max_features=5000)


# X_train_tfidf = tfidf.fit_transform(X_train_split)


# X_val_tfidf = tfidf.transform(X_val_split)  
# X_test_tfidf = tfidf.transform(test_data['clean_text']) 


# tfidf = TfidfVectorizer(max_features=5000)
# X_train = tfidf.fit_transform(train_data['clean_text'])
# X_test = tfidf.transform(test_data['clean_text'])

# tfidf = TfidfVectorizer(max_features=5000)
# X_train = tfidf.fit_transform(train_data['clean_text'])
# X_test = tfidf.transform(test_data['clean_text'])

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
# Using Bag of Words (CountVectorizer)
bow = CountVectorizer(max_features=5000)

In [17]:
X_train_bow = bow.fit_transform(X_train_split)
X_val_bow = bow.transform(X_val_split)
X_test_bow = bow.transform(test_data['clean_text'])

In [18]:
!pip install xgboost

In [19]:
# rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
# rf_classifier.fit(X_train_tfidf, y_train_split)

In [20]:
import xgboost as xgb

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [22]:
# Create a pipeline that combines Bag of Words (CountVectorizer) with the XGBoost classifier
pipeline = Pipeline([
    ('bow', CountVectorizer(max_features=5000)),  # Bag of Words
    ('xgb', xgb.XGBClassifier(objective='multi:softmax', num_class=6))  # XGBoost Classifier
])

In [23]:
from skopt import BayesSearchCV
search_space = {
    'xgb__eta': (0.01, 0.5, 'log-uniform'),  # logarithmic scale
    'xgb__max_depth': (3, 10),               # Integer values
    'xgb__n_estimators': (50, 300),
    'xgb__colsample_bytree': (0.5, 1.0)
}

In [24]:
bayes_search = BayesSearchCV(pipeline, search_space, n_iter=30, scoring='f1_weighted', cv=5)
bayes_search.fit(X_train_split, y_train_split)

BayesSearchCV(cv=5,
              estimator=Pipeline(steps=[('bow',
                                         CountVectorizer(max_features=5000)),
                                        ('xgb',
                                         XGBClassifier(base_score=None,
                                                       booster=None,
                                                       callbacks=None,
                                                       colsample_bylevel=None,
                                                       colsample_bynode=None,
                                                       colsample_bytree=None,
                                                       device=None,
                                                       early_stopping_rounds=None,
                                                       enable_categorical=False,
                                                       eval_metric=None,
                                                       feature_types=None,
                                                       gamma=None,
                                                       grow_policy=None,
                                                       im...
                                                       max_depth=None,
                                                       max_leaves=None,
                                                       min_child_weight=None,
                                                       missing=nan,
                                                       monotone_constraints=None,
                                                       multi_strategy=None,
                                                       n_estimators=None,
                                                       n_jobs=None, num_class=6,
                                                       num_parallel_tree=None, ...))]),
              n_iter=30, scoring='f1_weighted',
              search_spaces={'xgb__colsample_bytree': (0.5, 1.0),
                             'xgb__eta': (0.01, 0.5, 'log-uniform'),
                             'xgb__max_depth': (3, 10),
                             'xgb__n_estimators': (50, 300)})

In [25]:
# Get the best parameters from cross-validation
print("Best Parameters:", bayes_search.best_params_)

# Evaluate on the validation set
val_predictions_bayes = bayes_search.predict(X_val_split)

Best Parameters: OrderedDict([('xgb__colsample_bytree', 1.0), ('xgb__eta', 0.17554181819381467), ('xgb__max_depth', 10), ('xgb__n_estimators', 300)])


In [26]:
print("Validation Accuracy with Cross-Validation:", accuracy_score(y_val_split, val_predictions_bayes))
print("\nClassification Report on Validation Set with Cross-Validation:")
print(classification_report(y_val_split, val_predictions_bayes))

Validation Accuracy with Cross-Validation: 0.8626444159178434

Classification Report on Validation Set with Cross-Validation:
              precision    recall  f1-score   support

           0       0.71      0.54      0.61       165
           1       0.63      0.48      0.54       134
           2       0.72      0.26      0.38       178
           3       0.88      0.97      0.92      2312
           4       0.92      0.83      0.88       198
           5       0.83      0.67      0.74       129

    accuracy                           0.86      3116
   macro avg       0.78      0.62      0.68      3116
weighted avg       0.85      0.86      0.85      3116



In [27]:
# Make predictions on the test data using the best model from cross-validation
test_predictions_bayes = bayes_search.predict(test_data['clean_text'])

In [28]:
# Save the test predictions to a CSV file
test_data['Category'] = test_predictions_bayes
submission_bayes = test_data[['ID', 'Category']]
submission_bayes.to_csv('test_predictions_bayes.csv', index=False)

print("Test predictions with Bayesian optimization saved to 'test_predictions_bayes.csv'.")

Test predictions with Bayesian optimization saved to 'test_predictions_bayes.csv'.


# **Pre-Processing**

*sklearn library*

Missing values in the News_headline column are replaced with 'missing_headline' to ensure no NaN values interfere with model training.

Stopword Removal: Common words with little meaning in classification are removed using NLTK’s English stopwords list.

Lemmatization: Words are reduced to their root form using NLTK's WordNetLemmatizer, ensuring that words like "running" and "run" are treated as the same.

Regex-based Cleaning: Regular expressions are used to remove special characters and numbers from the text, leaving only alphabetic content.

Combining Fields: The News_title and News_headline are concatenated to form a combined text column, which is further cleaned using the preprocessing function.


# **Feature Engineering:**

Title and Headline Lengths: New features are created based on the word count of News_title and News_headline, adding more numerical data to potentially enhance the classification.

# **Label Encoding:**
The news categories (e.g., Arts, Business) are mapped to numerical values, making them suitable for classification.

# **Bag of Words (BoW) Feature Extraction:**
We first used tfidf but later on changed to bagofwords

CountVectorizer: We use a Bag of Words model to convert the textual data into a numerical matrix representation. The vocabulary is limited to 5000 words to manage feature dimensionality.

# **XGBoost Classifier:**
We first used RandomForestClassifier then later on changed.

XGBoost is an efficient gradient boosting algorithm used for classification.
Objective Function: The objective is set to 'multi:softmax', as this is a multi-class classification problem with six distinct categories.

# **Bayesian Optimization (BayesSearchCV):**
We first applied gridsearchCV but BayesSearch gave better and more accurate results
Purpose: We apply Bayesian optimization using BayesSearchCV from skopt to find the best hyperparameters for XGBoost. This optimization method efficiently explores the hyperparameter space, balancing exploration and exploitation.
Search Space: Parameters such as learning rate (eta), maximum depth (max_depth), number of estimators (n_estimators), and column sampling (colsample_bytree) are tuned.